In [30]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import os
from ultralytics import YOLO
fo.config.requirement_error_level = 2

In [19]:
recyclables = ["Bottle", "Book"]
hazardous = ["Mobile phone"]
kitchen = ["Banana", "Apple", "Orange"]
residual = ["Plastic bag", "Toilet paper", "Coffee cup"]
wanted_classes = recyclables + hazardous + kitchen + residual
parent_categories = {
    'Bottle': 'recyclables',
    'Book': 'recyclables',
    'Mobile phone': 'hazardous',
    'Banana': 'kitchen',
    'Apple': 'kitchen',
    'Orange': 'kitchen',
    'Plastic bag': 'residual',
    'Toilet paper': 'residual',
    'Coffee cup': 'residual',
}

# 只使用Fifty-One
- 主要用于推理和可视化
- 封装了模型+预处理+后处理
- 设计目的是在FiftyOne生态中使用

## 获取数据集

In [20]:
# dataset_train = fo.load_dataset("oi-trashes-train-1000")

## 获取预训练模型

In [21]:
# foz.list_zoo_models()

In [22]:
# model = foz.load_zoo_model("yolov8s-oiv7-torch")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\liangjiaqi\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [28]:
# model_path = foz.download_zoo_model("yolov8s-oiv7-torch")

Model 'yolov8s-oiv7-torch' is already downloaded


(<fiftyone.zoo.models.ZooModel at 0x1f650f01f70>,
 'C:\\Users\\liangjiaqi\\fiftyone\\__models__\\yolov8s-oiv7.pt')

## 执行预测

In [23]:
# dataset_train.apply_model(model, label_field="yolov8_pred")

 100% |███████████████| 1000/1000 [6.5m elapsed, 0s remaining, 2.5 samples/s]       


## 查看结果

In [26]:
# session = fo.launch_app(dataset_train)

# 使用微调参数

In [31]:
model = YOLO("yolov8s-oiv7.pt")

In [32]:
# 开始微调训练
results = model.train(
    data='./export_oi/dataset.yaml',           # 数据集配置
    epochs=50,                  # 训练轮数
    imgsz=640,                  # 图像尺寸
    batch=16,                   # 批次大小
    device='cpu',               # 使用CPU（如果有GPU改为0或[0,1,2,3]）
    workers=4,                  # 数据加载线程数
    lr0=0.01,                   # 初始学习率
    patience=10,                # 早停耐心值
    save=True,                  # 保存检查点
    pretrained=True,            # 使用预训练权重
    optimizer='auto',           # 自动选择优化器
    verbose=True,               # 显示训练详情
    project='garbage_detection',# 项目名称
    name='yolov8s_finetuned'    # 实验名称
)

Ultralytics 8.3.231  Python-3.12.8 torch-2.5.1 CPU (13th Gen Intel Core i5-1340P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./export_oi/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-oiv7.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=Tr

## 细致微调

In [35]:
def precise_garbage_detection_finetune(model_path, data_path):
    # 阶段1：只训练检测头（快速适应新类别）
    print("=== 阶段1: 检测头微调 (层22) ===")
    model = YOLO(model_path)
    model.train(
        data=data_path,
        epochs=15,
        imgsz=640,
        batch=16,
        lr0=0.01,
        freeze=22,  # 冻结前22层（骨干+颈部），只训练检测头
        patience=5,
        save=True,
        project='garbage_detection_precise',
        name='stage1_head_only',
        # 基础数据增强
        hsv_h=0.01,
        hsv_s=0.5, 
        hsv_v=0.3,
        translate=0.05,
        scale=0.2,
        fliplr=0.3,
    )
    
    # 阶段2：微调颈部网络（特征融合层）
    print("=== 阶段2: 颈部网络微调 (层10-21) ===")
    model = YOLO('garbage_detection_precise/stage1_head_only/weights/best.pt')
    model.train(
        data=data_path,
        epochs=20,
        imgsz=640,
        batch=16,
        lr0=0.005,
        freeze=10,  # 冻结前10层（骨干），训练颈部+头部
        patience=7,
        project='garbage_detection_precise',
        name='stage2_neck_tuning',
        # 增强数据增强
        mosaic=0.8,
        mixup=0.05,
        hsv_h=0.015,
        hsv_s=0.6, 
        hsv_v=0.4,
        translate=0.1,
        scale=0.3,
    )
    
    # 阶段3：微调深层骨干网络
    print("=== 阶段3: 深层骨干微调 (层5-9) ===")
    model = YOLO('garbage_detection_precise/stage2_neck_tuning/weights/best.pt')
    model.train(
        data=data_path,
        epochs=15,
        imgsz=640,
        batch=16,
        lr0=0.001,
        freeze=5,   # 冻结前5层（浅层骨干），训练深层骨干+颈部+头部
        patience=10,
        project='garbage_detection_precise',
        name='stage3_deep_backbone',
        # 更强数据增强
        mosaic=1.0,
        mixup=0.1,
        hsv_h=0.02,
        hsv_s=0.7, 
        hsv_v=0.5,
        translate=0.15,
        scale=0.4,
        fliplr=0.5,
    )
    
    # 阶段4：全网络精调（可选）
    print("=== 阶段4: 全网络精调 ===")
    model = YOLO('garbage_detection_precise/stage3_deep_backbone/weights/best.pt')
    model.train(
        data=data_path,
        epochs=10,
        imgsz=640,
        batch=16,
        lr0=0.0005,
        freeze=0,   # 解冻所有层
        patience=15,
        project='garbage_detection_precise',
        name='stage4_full_finetune',
        # 最强数据增强
        mosaic=1.0,
        mixup=0.15,
        copy_paste=0.1,
        hsv_h=0.025,
        hsv_s=0.8, 
        hsv_v=0.6,
        translate=0.2,
        scale=0.5,
    )
    
    return 'garbage_detection_precise/stage4_full_finetune/weights/best.pt'

In [36]:
final_model_path = precise_garbage_detection_finetune(model_path="yolov8s-oiv7.pt", data_path='./export_oi/dataset.yaml')

=== 阶段1: 检测头微调 (层22) ===
New https://pypi.org/project/ultralytics/8.3.232 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.231  Python-3.12.8 torch-2.5.1 CPU (13th Gen Intel Core i5-1340P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./export_oi/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=22, half=False, hsv_h=0.01, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-oiv7.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=stage1_head_only